In [99]:
#reload source files automatically
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
import pandas as pd

from event_clustering.preprocessing import *

# Loading Preprocessing Analyzing

In [39]:
file_name = 'DomesticDeclarations'
df = preprocess(load('data/' + file_name + '.xes'))
original_df_columns = df.columns

In [40]:
analyze(df)
df.head(10)

column name: case:Amount
data type: <class 'numpy.float64'>
unique values: 8326


column name: case:BudgetNumber
data type: <class 'str'>
unique values: 1


column name: case:DeclarationNumber
data type: <class 'str'>
unique values: 10049


column name: case:concept:name
data type: <class 'str'>
unique values: 10500


column name: case:id
data type: <class 'str'>
unique values: 10500


column name: concept:name
data type: <class 'str'>
unique values: 17


column name: id
data type: <class 'str'>
unique values: 56437


column name: org:resource
data type: <class 'str'>
unique values: 2


column name: org:role
data type: <class 'str'>
unique values: 7


column name: time:timestamp
data type: <class 'datetime.datetime'>
unique values: 45403


min case length: 1
max case length: 24
mean case length: 5.374952380952381


,case:Amount,case:BudgetNumber,case:DeclarationNumber,case:concept:name,case:id,concept:name,id,org:resource,org:role,time:timestamp
0,26.851205,budget 86566,declaration number 86792,declaration 86791,declaration 86791,Declaration SUBMITTED by EMPLOYEE,st_step 86794_0,STAFF MEMBER,EMPLOYEE,2017-01-09 09:49:50+01:00
1,182.464172,budget 86566,declaration number 86796,declaration 86795,declaration 86795,Declaration SUBMITTED by EMPLOYEE,st_step 86798_0,STAFF MEMBER,EMPLOYEE,2017-01-09 10:26:14+01:00
2,320.646137,budget 86566,declaration number 86801,declaration 86800,declaration 86800,Declaration SUBMITTED by EMPLOYEE,st_step 86804_0,STAFF MEMBER,EMPLOYEE,2017-01-09 11:13:33+01:00
3,113.862047,budget 86566,declaration number 86732,declaration 86731,declaration 86731,Declaration SUBMITTED by EMPLOYEE,st_step 86733_0,STAFF MEMBER,EMPLOYEE,2017-01-09 11:24:20+01:00
4,26.851205,budget 86566,declaration number 86792,declaration 86791,declaration 86791,Declaration FINAL_APPROVED by SUPERVISOR,st_step 86793_0,STAFF MEMBER,SUPERVISOR,2017-01-09 11:27:48+01:00
5,134.404169,budget 86566,declaration number 86736,declaration 86735,declaration 86735,Declaration SUBMITTED by EMPLOYEE,st_step 86738_0,STAFF MEMBER,EMPLOYEE,2017-01-09 11:30:26+01:00
6,134.404169,budget 86566,declaration number 86736,declaration 86735,declaration 86735,Declaration FINAL_APPROVED by SUPERVISOR,st_step 86737_0,STAFF MEMBER,SUPERVISOR,2017-01-09 11:37:18+01:00
7,113.862047,budget 86566,declaration number 86732,declaration 86731,declaration 86731,Declaration FINAL_APPROVED by SUPERVISOR,st_step 86734_0,STAFF MEMBER,SUPERVISOR,2017-01-09 11:37:23+01:00
8,443.585163,budget 86566,declaration number 86806,declaration 86805,declaration 86805,Declaration SUBMITTED by EMPLOYEE,st_step 86807_0,STAFF MEMBER,EMPLOYEE,2017-01-09 11:40:47+01:00
9,443.585163,budget 86566,declaration number 86806,declaration 86805,declaration 86805,Declaration FINAL_APPROVED by SUPERVISOR,st_step 86808_0,STAFF MEMBER,SUPERVISOR,2017-01-09 11:41:23+01:00


In [97]:
# Todo column name mapping

# Feature Generation

In [41]:
add_event_type_representative(df)

In [ ]:
add_event_ref(df, 1)
add_event_ref(df, -1)
determine_event_position(df)

In [300]:
add_timestamp_features(df)

In [101]:
df.to_csv('data/' + file_name +'_features.csv', index=False)

# Encoding

In [102]:
df = preprocess(pd.read_csv('data/' + file_name +'_features.csv'))

In [111]:
df_name = tfidf_encode(df, 'concept:name', TfidfVectorizer(stop_words = 'english'))
df_role = one_hot_encode(df, 'org:role')
df_event_position = df[['feature_pos_beginning', 'feature_pos_middle', 'feature_pos_end']]
df_time = binning(df, 'event_ref_time_1', 4)

In [114]:
df_encoded = df_name
df_encoded = df_encoded.join(df_role)
df_encoded = df_encoded.join(df_event_position)
#df_encoded = df_encoded.join(df['feature_day_nr'])

In [115]:
# file_name_structure: [Dataset-name]_[Feature-groups]_encoded.csv
df_encoded.to_csv('data/' + file_name +'_name-role-position_encoded.csv', index=False)